## Cleaning the text and pre processing 


In [3]:
#functio to load text document
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [4]:
in_filename = 'republic.txt'
doc = load_doc(in_filename)
print(doc[:200])

ï»¿ BOOK I.


I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in


In [5]:
import string
# clean the loaded document remove punctuations and one letter words and return it as tokens 
def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [6]:
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

In [7]:
# create sequences out of these tokens by selecting 51 tokens of which first 50 will be used as input and next 1 as output  
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 117291


In [8]:
# a function to save these sequences in a text file
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [9]:
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

# Training The model

In [10]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [11]:
from keras.preprocessing.text import Tokenizer

# Encdiing the textual data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)# fit the tokenizer to text
sequences = tokenizer.texts_to_sequences(lines) # convert the given text to integer sequence 

In [14]:
vocab_size = len(tokenizer.word_index) + 1 # we take vocabulary size as vocabulary size + 1 because tokenizer return a 1 indexed tokens

In [15]:
import numpy as np
from keras.utils import to_categorical

sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size) # one hot encode on the basis of 51st word 
seq_length = X.shape[1]

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


model = keras.Sequential() # creating a sequential model
model.add(layers.Embedding(input_dim=vocab_size, output_dim =  50, input_length=seq_length)) # turns +ve indexes to dense vectors
model.add(layers.LSTM(100, return_sequences=True)) # two LSTM layers of 100 neurons each
model.add(layers.LSTM(100))
model.add(layers.Dense(100, activation='relu')) # redense layer to reduce losses while learning 
model.add(layers.Dense(vocab_size, activation='softmax')) # dense 
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            366200    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7324)              739724    
                                                                 
Total params: 1,256,824
Trainable params: 1,256,824
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
917/917 [==============================] - 149s 159ms/step - loss: 6.1475 - accuracy: 0.0735
Epoch 2/100
917/917 [==============================] - 140s 153ms/step - loss: 5.6910 - accuracy: 0.1054
Epoch 3/100
917/917 [==============================] - 141s 154ms/step - loss: 5.4575 - accuracy: 0.1281
Epoch 4/100
917/917 [==============================] - 143s 156ms/step - loss: 5.2914 - accuracy: 0.1454
Epoch 5/100
917/917 [==============================] - 145s 158ms/step - loss: 5.1702 - accuracy: 0.1534
Epoch 6/100
917/917 [==============================] - 144s 157ms/step - loss: 5.0734 - accuracy: 0.1604
Epoch 7/100
917/917 [==============================] - 144s 157ms/step - loss: 4.9862 - accuracy: 0.1659
Epoch 8/100
917/917 [==============================] - 147s 160ms/step - loss: 4.9285 - accuracy: 0.1685
Epoch 9/100
917/917 [==============================] - 146s 159ms/step - loss: 4.8353 - accuracy: 0.1755
Epoch 10/100
917/917 [==============================] -

In [18]:
# saving the model
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

model.save('model_republic.h5')
dump(tokenizer, open('tokenizer_republic.pkl', 'wb'))